In [56]:
import torch
from torch import nn

In [57]:
def corr2d(X,K):
    h,w = K.shape
    Y = torch.zeros((X.shape[0] -h+1), (X.shape[1] -w +1))
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            mul = (X[i:i+h, j:j+w] * K)
#             print(mul)
            Y[i,j] = mul.sum()
    
    return Y

In [58]:
def corr2d(X, K):  #@save
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [59]:
X = torch.tensor([[1.0,2.0,3.0], [2.0,3.0,4.0],[3.0,4.0,5.0]])
K = torch.tensor([[1.0,2.0], [2.0,3.0]])

corr2d(X,K)

tensor([[18., 26.],
        [26., 34.]])

In [60]:
class Conv2d(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

In [61]:
net = Conv2d(K.shape)

In [62]:
net(X)

tensor([[-2.4563, -4.4002],
        [-4.4002, -6.3442]], grad_fn=<AddBackward0>)

### Edge detection

In [63]:
X = torch.ones((8,8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [64]:
K = torch.tensor([[1.0,-1.0]])

In [65]:
corr2d(X,K)
# detects horizontal edges

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [66]:
corr2d(X.t(),K)
# but not vertical edges

tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])

In [67]:
X = torch.ones((8,8))

X[:,2:3] = 0
X[:,4:6] = 0

print(X)
Y = corr2d(X,K)
Y, Y.shape
# interesting

tensor([[1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.]])


(tensor([[ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.]]),
 torch.Size([8, 7]))

In [68]:
K.shape

torch.Size([1, 2])

### Learning a convolution

In [69]:
X = torch.ones((6,8))
X[:,2:6] = 0

Y =  corr2d(X,K)

In [70]:
conv2d = nn.Conv2d(1,1,kernel_size=(1,2), bias=False)

X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
lr = 3e-2

print(X.shape)
print(Y.shape)

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    
    if (i+1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum()}')


torch.Size([1, 1, 6, 8])
torch.Size([1, 1, 6, 7])
batch 2, loss 11.608417510986328
batch 4, loss 2.581953287124634
batch 6, loss 0.6930234432220459
batch 8, loss 0.22270187735557556
batch 10, loss 0.08095771074295044


In [71]:
conv2d.weight.data

tensor([[[[ 1.0125, -0.9563]]]])

### Exercises

1. Construct an image X with diagonal edges.
    1. What happens if you apply the kernel K in this section to it?
        * zero matrix.
    2. What happens if you transpose X?
        * No change
    3. What happens if you transpose K?
        * zero matrix.
        
2. When you try to automatically find the gradient for the Conv2D class we created, what kind
of error message do you see?
    * I am able to do `net.weights.grad`, when I try `net.grad` I get the error `'Conv2d' object has no attribute 'grad'`

3. How do you represent a cross-correlation operation as a matrix multiplication by changing
the input and kernel tensors?
    * cross correlation is basically matrix multiplication between slices of tensorfrom X of the shape of kernel and summing.
    * It can be done by padding Kand X based on what is needed to multiply

4. Design some kernels manually.
    1. What is the form of a kernel for the second derivative?
        * okay in order to compute one way would be to manually compute the second derivative and then let see a kernel be made using backpropogation
    2. What is the kernel for an integral?
        * how do you actually make it manually
3. What is the minimum size of a kernel to obtain a derivative of degree d
        * dont know.

In [72]:
#1

X =  torch.zeros((8,8))
for i in range(X.shape[0]):
    X[i][i] = 1
X

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])

In [73]:
K = torch.tensor([[1.0,-1.0]])
K.shape

torch.Size([1, 2])

In [74]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (Y[i:i+h, j : j +w] * K).sum()
    return Y

In [75]:
corr2d(X, K)

tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])

In [76]:
X.t(), X

(tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1.]]),
 tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1.]]))

In [77]:
corr2d(X, K.t())

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [78]:
Y = corr2d(X, K)
Y.shape

torch.Size([8, 7])

In [79]:
net = nn.Conv2d(1,1,K.shape, bias=False)

X = X.reshape(1,1,8,8)
Y = Y.reshape(1,1,8,7)

print(Y, X)
lr = 3e-2

for i in range(10):
    y_hat = net(X)
#     print(y_hat.shape)
    l = (y_hat - Y)**2
    
    net.zero_grad()
    
    l.sum().backward()
    
    net.weight.data -= lr * net.weight.grad
    
    print(f"for epoch {i} loss: {l.sum()}")


tensor([[[[0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.]]]]) tensor([[[[1., 0., 0., 0., 0., 0., 0., 0.],
          [0., 1., 0., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 0., 0., 0., 1.]]]])
for epoch 0 loss: 0.17383873462677002
for epoch 1 loss: 0.05847935751080513
for epoch 2 loss: 0.0196724571287632
for epoch 3 loss: 0.006617813836783171
for epoch 4 loss: 0.0022262325510382652
for epoch 5 loss: 0.0007489045965485275
for epoch 6 loss: 0.0002519315166864544
for epoch 7 loss: 8.474975766148418e-05
for epoch 

In [80]:
print(net.weight.data[0][0])
corr2d(X, net.weight.data[0][0])

tensor([[0.0006, 0.0003]])


tensor([], size=(1, 0))

In [81]:
try:
    net.grad
except Exception as e:
    print(e)

'Conv2d' object has no attribute 'grad'


### Strides and padding

In [82]:
import torch
from torch import nn

In [83]:
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [84]:
(1,1)+ X.shape

(1, 1, 6, 8)

In [85]:
Z = X.reshape((1,1) +X.shape)
Z
# 1 ,1 we add for batch and channel

tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.]]]])

In [86]:
conv2d(Z)

tensor([[[[ 0.0561,  1.0125,  0.0000,  0.0000,  0.0000, -0.9563,  0.0561],
          [ 0.0561,  1.0125,  0.0000,  0.0000,  0.0000, -0.9563,  0.0561],
          [ 0.0561,  1.0125,  0.0000,  0.0000,  0.0000, -0.9563,  0.0561],
          [ 0.0561,  1.0125,  0.0000,  0.0000,  0.0000, -0.9563,  0.0561],
          [ 0.0561,  1.0125,  0.0000,  0.0000,  0.0000, -0.9563,  0.0561],
          [ 0.0561,  1.0125,  0.0000,  0.0000,  0.0000, -0.9563,  0.0561]]]],
       grad_fn=<ThnnConv2DBackward>)

In [87]:
def comp_conv2d(conv2d, X):
    X = X.reshape((1,1)+ X.shape)
    Y = conv2d(X)
    # we dont need batch and channels
    return Y.reshape(Y.shape[2:])

In [88]:
conv2d= nn.Conv2d(1,1,kernel_size=3, padding=1)
X = torch.rand(size=(8,8))
comp_conv2d(conv2d, X).shape
# note that here since padding 1 means to either side 

torch.Size([8, 8])

In [89]:
conv2d= nn.Conv2d(1,1,kernel_size=(5,3), padding=(2,1))


comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [90]:
conv2d = nn.Conv2d(1,1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [91]:
conv2d = nn.Conv2d(1,1, kernel_size=(5,3), padding=(2,1), stride=2)
print(X.shape)
comp_conv2d(conv2d, X).shape

# it halves the input

torch.Size([8, 8])


torch.Size([4, 4])

In [92]:
conv2d = nn.Conv2d(1,1, kernel_size=(3,5), padding=(0,1), stride=(3,4))
conv2d(X.reshape((1,1)+ X.shape)).shape

torch.Size([1, 1, 2, 2])

### Exercises
1. For the last example in this section, use mathematics to calculate the output shape to see if
it is consistent with the experimental result.

* it is consistent, |(8 -3 + 0 + 3)/3| , |(8-5+1+4)/4|

2. Try other padding and stride combinations on the experiments in this section.

*  hmm tried

3. For audio signals, what does a stride of 2 correspond to?

* it might be two time peridod long

4. What are the computational benefits of a stride larger than 1

* efficiency in calculation,downsampling

## Multiple input and output channels

In [93]:
#creating cross correlation for multiple channels
def corr2d_multi_in_my(X, K):
    sum_Y = 0
    for x, k in zip(X,K):
        nh, nw = x.shape
        h, w = k.shape
        Y = torch.zeros((nh-h+1), (nw-w+1))
        for i in range(Y.shape[0]):
            for j in range(Y.shape[1]):
                x_in = x[i:i+h, j:j+w]
#                 print(x_in.shape, k.shape)
                Y[i][j] = ( x_in * k).sum()
    print(Y)
    sum_Y += Y
    return sum_Y
                
    

In [94]:
def corr2d_multi_in(X, K):
    return sum(corr2d(x,k) for x, k in zip(X,K))

In [95]:
def corr2d_multi_in(X, K):
    # First, iterate through the 0th dimension (channel dimension) of `X` and
    # `K`. Then, add them together
    return sum(corr2d(x, k) for x, k in zip(X, K))

In [96]:
K = torch.tensor([[[1,2],[3,4]], [[4,5],[5,6]]])
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])

In [97]:
X.shape

torch.Size([2, 3, 3])

In [98]:
K.shape

torch.Size([2, 2, 2])

In [99]:
X

tensor([[[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]])

In [100]:
K

tensor([[[1, 2],
         [3, 4]],

        [[4, 5],
         [5, 6]]])

In [101]:
for k in K:
    print(k)

tensor([[1, 2],
        [3, 4]])
tensor([[4, 5],
        [5, 6]])


In [102]:
for x in X:
    print(x)

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])


In [103]:
for x, k in zip(X, K):
    print(x)
    print(k)
    print("\n")

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])
tensor([[1, 2],
        [3, 4]])


tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
tensor([[4, 5],
        [5, 6]])




In [104]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

In [105]:
corr2d_multi_in(X,K)

tensor([[0., 0.],
        [0., 0.]])

In [106]:
corr2d_multi_in_my(X, K)

tensor([[37., 47.],
        [67., 77.]])


tensor([[37., 47.],
        [67., 77.]])

In [109]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in_my(X,K) for k in K], 0)

In [111]:
res = corr2d_multi_in_out(X,K)
print(res)
print(res.shape)

tensor([[37., 47.],
        [67., 77.]])
tensor([[37., 47.],
        [67., 77.]])
tensor([[[37., 47.],
         [67., 77.]],

        [[37., 47.],
         [67., 77.]]])
torch.Size([2, 2, 2])


In [112]:
# Weconstruct a convolution kernel with 3 output channel by concatenating the kernel tensor K with K+1
K= torch.stack((K,K+1, K+2),0)
K.shape

torch.Size([3, 2, 2, 2])

In [114]:
def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of `K`, and each time, perform
    # cross-correlation operations with input `X`. All of the results are
    # stacked together
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [115]:
res = corr2d_multi_in_out(X,K)
print(res)
print(res.shape)

tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]])
torch.Size([3, 2, 2])


You could think of the 1 × 1 convolutional layer as constituting a fully-connected
layer applied at every single pixel location to transform the ci corresponding input values into co
output values. Because this is still a convolutional layer, the weights are tied across pixel location.
Thus the 1 × 1 convolutional layer requires co × ci weights (plus the bias).

### 1x1 convolutional layer

In [118]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K,X)
    return Y.reshape(c_o, h, w)

In [120]:
X = torch.normal(0,1,(3,3,3))
K = torch.normal(0,1,(2,3,1,1))

Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
print(float(torch.abs(Y1-Y2).sum()) )


9.801506996154785
